In [ ]:
from game import Directions
from game import Agent
from game import Actions
import util
import time
import search

class GoWestAgent(Agent):
    def getAction(self, state):
        if Directions.WEST in state.getLegalPacmanActions():
            return Directions.WEST
        else:
            return Directions.STOP

class SearchAgent(Agent):
  def __init__(self, fn='depthFirstSearch', prob='PositionSearchProblem', heuristic='nullHeuristic'):
    if fn not in dir(search):
      raise AttributeError, fn + ' is not a search function in search.py.'
    func = getattr(search, fn)
    if 'heuristic' not in func.func_code.co_varnames:
      print('[SearchAgent] using function ' + fn)
      self.searchFunction = func
    else:
      if heuristic in globals().keys():
        heur = globals()[heuristic]
      elif heuristic in dir(search):
        heur = getattr(search, heuristic)
      else:
        raise AttributeError, heuristic + ' is not a function in searchAgents.py or search.py.'
      print('[SearchAgent] using function %s and heuristic %s' % (fn, heuristic))
      self.searchFunction = lambda x: func(x, heuristic=heur)

    if prob not in globals().keys() or not prob.endswith('Problem'):
      raise AttributeError, prob + ' is not a search problem type in SearchAgents.py.'
    self.searchType = globals()[prob]
    print('[SearchAgent] using problem type ' + prob)

  def registerInitialState(self, state):
    if self.searchFunction == None: raise Exception, "No search function provided for SearchAgent"
    starttime = time.time()
    problem = self.searchType(state)
    self.actions  = self.searchFunction(problem)
    totalCost = problem.getCostOfActions(self.actions)
    print('Path found with total cost of %d in %.1f seconds' % (totalCost, time.time() - starttime))
    if '_expanded' in dir(problem): print('Search nodes expanded: %d' % problem._expanded)

  def getAction(self, state):
    if 'actionIndex' not in dir(self): self.actionIndex = 0
    i = self.actionIndex
    self.actionIndex += 1
    if i < len(self.actions):
      return self.actions[i]
    else:
      return Directions.STOP

class PositionSearchProblem(search.SearchProblem):
  def __init__(self, gameState, costFn = lambda x:1, goal = (1,1), start=None, warn=True, visualize = True):
    self.walls = gameState.getWalls()
    self.startState = gameState.getPacmanPosition()
    if start != None: self.startState = start
    self.goal = goal
    self.costFn = costFn
    self.visualize = visualize
    if warn and (gameState.getNumFood() != 1 or not gameState.hasFood(*goal)):
      print 'Warning: this does not look like a regular search maze'

    self._visited, self._visitedlist, self._expanded = {}, [], 0

  def getStartState(self):
    return self.startState

  def isGoalState(self, state):
    isGoal = state == self.goal

    if isGoal and self.visualize:
      self._visitedlist.append(state)
      import __main__
      if '_display' in dir(__main__):
        if 'drawExpandedCells' in dir(__main__._display):
          __main__._display.drawExpandedCells(self._visitedlist)

    return isGoal

  def getSuccessors(self, state):
    successors = []
    for action in [Directions.NORTH, Directions.SOUTH, Directions.EAST, Directions.WEST]:
      x,y = state
      dx, dy = Actions.directionToVector(action)
      nextx, nexty = int(x + dx), int(y + dy)
      if not self.walls[nextx][nexty]:
        nextState = (nextx, nexty)
        cost = self.costFn(nextState)
        successors.append((nextState, action, cost))

    self._expanded += 1
    if state not in self._visited:
      self._visited[state] = True
      self._visitedlist.append(state)

    return successors

  def getCostOfActions(self, actions):

    if actions == None: return 999999
    x, y = sef.getStartState()
    cost = 0
    for action in actions:
      dx, dy = Actions.directionToVector(action)
      x, y = int(x + dx), int(y + dy)
      if self.walls[x][y]: return 999999
      cost += self.costFn((x,y))
    return cost

class StayEastSearchAgent(SearchAgent):
  def __init__(self):
    self.searchFunction = search.uniformCostSearch
    costFn = lambda pos: .5 ** pos[0]
    self.searchType = lambda state: PositionSearchProblem(state, costFn, (1,1), None, False)

class StayWestSearchAgent(SearchAgent):
  def __init__(self):
    self.searchFunction = search.uniformCostSearch
    costFn = lambda pos: .5 ** pos[0]
    self.searchType = lambda state: PositionSearchProblem(state, costFn)

def manhattanHeuristic(position, problem, info={}):
  xy1 = position
  xy2 = problem.goal
  return abs(xy1[0] - xy2[0]) + abs(xy1[1] - xy2[1])

def euclideanHeuristic(position, problem, info={}):
  xy1 = position
  xy2 = problem.goal
  return ((xy1[0] - xy2[0]) ** 2 + (xy1[1] - xy2[1]) ** 2) ** 0.5